# Lab 12: Grid Localization using Bayes Filter (Real Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

### <span style="color:rgb(0,150,0)">The notebook only provides skeleton code for you to integrate the Localization class with the Real Robot.</span>

<hr>

In [19]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
import pathlib
import os
from utils import load_config_params
from localization_extras import Localization

# The imports below will only work if you copied the required ble-related python files 
# into the notebooks directory
from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import numpy as np

# Lab 9 imports
from funcs import *
from robotClass import *

# Setup Logger
LOG = get_logger('demo_notebook.log')
LOG.propagate = False

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2022-05-20 00:19:26,764 | INFO     |: Logger demo_notebook.log already initialized.
2022-05-20 00:19:26,772 | INFO     |: GUI is already running. Shutdown notebook to force restart the GUI.


In [20]:
# Start the plotter
START_PLOTTER()

2022-05-20 00:19:27,674 | INFO     |: Creating New Plotter Process


# Data from robot runs

In [ ]:
# Data from multiple trials at each location
loc_0_3 = {
    0: [0.605, 0.605, 0.558, 0.554, 0.414, 0.414, 0.358, 0.358, 0.35, 0.35, 0.634, 0.634, 0.673, 0.673, 0.492, 0.492, 0.594, 0.594],
    1: [0.526, 0.526, 0.501, 0.597, 0.597, 0.471, 0.471, 0.376, 0.376, 0.47, 0.47, 0.671, 0.671, 0.444, 0.444, 0.657, 0.657, 0.576],
    2: [0.422, 0.422, 0.721, 0.683, 0.551, 0.404, 0.404, 0.388, 0.388, 0.398, 0.398, 0.664, 0.664, 0.77, 0.77, 0.779, 0.779, 0.8],
    3: [0.785, 0.802, 0.688, 0.5, 0.378, 0.378, 0.393, 0.393, 0.412, 0.412, 0.638, 0.638, 0.732, 0.732, 0.708, 0.708, 0.75, 0.75]
}

loc_5_3 = {
    0: [0.413, 0.413, 0.412, 0.458, 0.565, 0.565, 0.45, 0.45, 0.455, 0.455, 0.682, 0.682, 0.747, 0.747, 0.493, 0.493, 0.71, 0.71],
    1: [0.474, 0.417, 0.486, 0.56, 0.466, 0.466, 0.438, 0.438, 0.62, 0.62, 0.853, 0.853, 0.6, 0.6, 0.593, 0.593, 0.644, 0.644]
}

loc_5_minus3 = {
    0: [0.701, 0.701, 0.392, 0.444, 0.582, 0.609, 0.609, 0.598, 0.598, 0.596, 0.596, 0.504, 0.504, 0.69, 0.69, 0.433, 0.433, 0.486],
    1: [0.416, 0.408, 0.425, 0.517, 0.646, 0.646, 0.502, 0.502, 0.689, 0.689, 0.413, 0.413, 0.853, 0.853, 0.505, 0.505, 0.446, 0.446]
}

loc_minus3_minus2 = {
    0: [0.556, 0.435, 0.504, 0.617, 0.465, 0.472, 0.563, 0.563, 0.609, 0.609, 0.716, 0.716, 0.685, 0.685, 0.825, 0.825, 0.793, 0.793],
    1: [0.566, 0.386, 0.674, 0.479, 0.46, 0.63, 0.655, 0.655, 0.768, 0.768, 0.658, 0.658, 0.667, 0.667, 0.729, 0.729, 0.725, 0.725],
    2: [0.564, 0.564, 0.415, 0.608, 0.462, 0.556, 0.6, 0.6, 0.628, 0.756, 0.756, 0.673, 0.673, 0.667, 0.667, 0.806, 0.806, 0.766]
}

In [27]:
n3n2_tof = [(655.0, 166.56714582443237), (723.0, 167.82818984985352), (919.0, 169.08666062355042), (719.0, 170.3430426120758), (659.0, 171.66160011291504), (681.0, 172.98369193077087), (863.0, 174.06581830978394), (858.0, 175.14470434188843), (696.0, 176.28691744804382), (712.0, 177.36669301986694), (849.0, 178.566876411438), (890.0, 179.58805203437805), (1221.0, 180.90769481658936), (3024.0, 182.16811084747314), (1919.0, 183.36635780334473), (2125.0, 184.56875205039978), (2162.0, 185.76617431640625), (671.0, 186.7842662334442), (673.0, 188.10858964920044)]

In [28]:
y = [val[0] for val in n3n2_tof]
x = y[2:] + y[:1]
x

[919.0,
 719.0,
 659.0,
 681.0,
 863.0,
 858.0,
 696.0,
 712.0,
 849.0,
 890.0,
 1221.0,
 3024.0,
 1919.0,
 2125.0,
 2162.0,
 671.0,
 673.0,
 655.0]

In [ ]:
p5n3_tof = [(2351.0, 12.130674362182617), (2222.0, 13.209893226623535), (808.0, 14.33945918083191), (1376.0, 15.30992579460144), (3070.0, 16.3868727684021), (2232.0, 17.34799885749817), (945.0, 18.430388689041138), (485.0, 19.32319164276123), (358.0, 20.344916820526123), (340.0, 21.30400848388672), (377.0, 22.26934242248535), (518.0, 23.281633377075195), (482.0, 24.365478992462158), (394.0, 25.509005546569824), (396.0, 26.46590518951416), (465.0, 27.73127055168152), (819.0, 28.6215980052948), (1953.0, 29.714779138565063), (2005.0, 30.663822174072266)]

In [3]:
p5p3_tof = [(384.0, 396.39471197128296), (425.0, 397.47551369667053), (560.0, 398.4272940158844), (1204.0, 399.3923830986023), (2505.0, 400.35790038108826), (2995.0, 401.43852162361145), (621.0, 402.6385006904602), (466.0, 403.648610830307), (2138.0, 404.67747044563293), (1835.0, 405.93231773376465), (568.0, 406.95194029808044), (421.0, 407.8569657802582), (359.0, 408.93391823768616), (347.0, 409.89200019836426), (406.0, 410.91195797920227), (535.0, 411.818243265152), (437.0, 412.77171659469604), (400.0, 413.67212891578674)]

In [4]:
y = [val[0] for val in p5p3_tof]
x = y[-10:] + y[:-10]

In [13]:
zp3_tof = [(365.0, 34.042689085006714), (367.0, 34.82365369796753), (399.0, 35.60405969619751), (519.0, 36.37975811958313), (738.0, 37.220059871673584), (659.0, 38.183263540267944), (728.0, 38.96185493469238), (943.0, 39.804969787597656), (2595.0, 40.57909059524536), (2199.0, 41.366214752197266), (2544.0, 42.260695457458496), (1146.0, 43.15843152999878), (928.0, 43.9950532913208), (2047.0, 44.96620416641235), (1963.0, 45.92703890800476), (655.0, 46.70462465286255), (457.0, 47.53348731994629), (381.0, 48.44501256942749)]

In [14]:
y = [val[0] for val in zp3_tof]
x = y[-5:] + y[:-5]

In [15]:
x

[2047.0,
 1963.0,
 655.0,
 457.0,
 381.0,
 365.0,
 367.0,
 399.0,
 519.0,
 738.0,
 659.0,
 728.0,
 943.0,
 2595.0,
 2199.0,
 2544.0,
 1146.0,
 928.0]

# The RealRobot class
Define the RealRobot class in the code cell below, based on the documentation and your real robot communication protocol. <br>
This class is used by the **Localization** class to communicate with the real robot. <br>
More specifically, the **Localization** class utilizes the **RealRobot's** member function **perform_observation_loop()** to get the 18 sensor readings and store them in its member variable **obs_range_data**, which is then utilized in the update step.

In [29]:
class RealRobot():
    """A class to interact with the real robot
    """
    def __init__(self, commander, ble):
        # Load world config
        self.world_config = os.path.join(str(pathlib.Path(os.getcwd()).parent), "config", "world.yaml")
        
        self.config_params = load_config_params(self.world_config)
        
        # Commander to commuincate with the Plotter process
        # Used by the Localization module to plot odom and belief
        self.cmdr = commander

        # ArtemisBLEController to communicate with the Robot
        self.ble = ble
        
        # Robot Control attribute
        self.rc = RobotControl(ble)

    def get_pose(self):
        """Get robot pose based on odometry
        
        Returns:
            current_odom -- Odometry Pose (meters, meters, degrees)
        """
        raise NotImplementedError("get_pose is not implemented")

    async def perform_observation_loop(self, rot_vel=120):
        """Perform the observation loop behavior on the real robot, where the robot does  
        a 360 degree turn in place while collecting equidistant (in the angular space) sensor
        readings, with the first sensor reading taken at the robot's current heading. 
        The number of sensor readings depends on "observations_count"(=18) defined in world.yaml.
        
        Keyword arguments:
            rot_vel -- (Optional) Angular Velocity for loop (degrees/second)
                        Do not remove this parameter from the function definition, even if you don't use it.
        Returns:
            sensor_ranges   -- A column numpy array of the range values (meters)
            sensor_bearings -- A column numpy array of the bearings at which the sensor readings were taken (degrees)
                               The bearing values are not used in the Localization module, so you may return a empty numpy array
        """
        #self.rc.pingRobot(clear = True) # clear attributes
        #self.rc.turn360(forwardSpeed = 140, backwardSpeed = 140, dir_ = 0)
        #await asyncio.sleep(5)
        
        sensorVals = np.array(x).reshape(18, 1)/1000#np.array(loc_0_3[0]).reshape(18, 1)
        #sensorVals = np.ones(18).reshape(18, 1)
        #sensorVals = np.ones(18).reshape(18, 1)
        gyroVals = np.array([2974.44995117, 2974.45556641, 2993.55810547, 3012.73193359, 3031.74584961, 
                             3050.94506836, 3070.16479492, 3089.19018555, 3108.37109375, 3127.76220703,
                             3147.09936523, 3166.15039062, 3185.69360352, 3204.91992188, 3224.35595703,
                             3243.38232422, 3262.41137695, 3281.63598633]).reshape(18, 1)
        
        
        #sensorVals = np.array( [ val[0] / 1000 for val in self.rc.tof2_readings] ).reshape(18, 1)
        #gyroVals = np.array( [ val[0] for val in self.rc.imu_readings] ).reshape(18, 1)
        
        print(sensorVals)
        print(gyroVals)
        
        # RETURN DISTANCES AND ANGLES
        return sensorVals, gyroVals

In [ ]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

In [30]:
# Initialize RealRobot with a Commander object to communicate with the plotter process
# and the ArtemisBLEController object to communicate with the real robot
ble = 5
robot = RealRobot(cmdr, ble)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a RealRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2022-05-20 00:22:20,685 | INFO     |:  | Number of observations per grid cell: 18
2022-05-20 00:22:20,687 | INFO     |:  | Precaching Views...
2022-05-20 00:22:25,550 | INFO     |:  | Precaching Time: 4.862 secs
2022-05-20 00:22:25,552 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-05-20 00:22:25,553 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


### Lab 12 Testing

In [ ]:
robot = RealRobot(cmdr, ble)

In [ ]:
x = await robot.perform_observation_loop()

In [ ]:
x

# Run an update step of the Bayes Filter

In [31]:
# Reset Plots
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
await loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.plot_update_step_data(plot_data=True)

# Plot GT
cmdr.plot_gt(-900/1000, -600/1000)

# Plot Odom and GT
#current_odom = robot.get_pose()
# cmdr.plot_gt(current_gt[0], current_gt[1])
#cmdr.plot_odom(current_odom[0], current_odom[1])

2022-05-20 00:22:27,837 | INFO     |: Initializing beliefs with a Uniform Distribution
2022-05-20 00:22:27,838 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
[[0.919]
 [0.719]
 [0.659]
 [0.681]
 [0.863]
 [0.858]
 [0.696]
 [0.712]
 [0.849]
 [0.89 ]
 [1.221]
 [3.024]
 [1.919]
 [2.125]
 [2.162]
 [0.671]
 [0.673]
 [0.655]]
[[2974.44995117]
 [2974.45556641]
 [2993.55810547]
 [3012.73193359]
 [3031.74584961]
 [3050.94506836]
 [3070.16479492]
 [3089.19018555]
 [3108.37109375]
 [3127.76220703]
 [3147.09936523]
 [3166.15039062]
 [3185.69360352]
 [3204.91992188]
 [3224.35595703]
 [3243.38232422]
 [3262.41137695]
 [3281.63598633]]
2022-05-20 00:22:27,844 | INFO     |: Update Step
2022-05-20 00:22:27,865 | INFO     |:      | Update Time: 0.018 secs
2022-05-20 00:22:27,871 | INFO     |: Bel index     : (2, 2, 16) with prob = 1.0
2022-05-20 00:22:27,879 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2022-05-20 00:22:27,886 | INFO     |: Belief        : (-0.914, -0.

### Plotting

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.title('Front ToF Sensor Readings')
plt.scatter([x for x in range(1, 19)], loc_0_3[1])
plt.xticks(np.arange(0, 20, 2))
plt.xlabel('Sensor Reading Number')
plt.ylabel('Sensor Value (m)')
plt.savefig('sensorReadings_0_3.png')

In [ ]:
plt.title('Front ToF Sensor Readings')
plt.scatter([x for x in range(1, 19)], loc_minus3_minus2[1])
plt.xticks(np.arange(0, 20, 2))
plt.xlabel('Sensor Reading Number')
plt.ylabel('Sensor Value (m)')
plt.savefig('sensorReadings_minus3_minus2.png')